# Assignment : Spam Filter
## Description
This assignment is near-final modulo some small adjustments (6 Nov '16)
In this assignment, you will discover that in many practical machine learning problems implementing the learning algorithm is often only a small part of the overall system. Thus to get a high mark for this assignment you need to implement any of the more advanced classification techniques or clever pre-processing methods. You will find plenty of them on the internet. If you do not know where to look for them, ask Google ;-).

Here your task is to build the standard (i.e. multinomial) Naive Bayes text classifier described during the lectures. You should test your program using the automatic marking software (described below), so it is critically important that it follows the specifications in detail.

You will train your classifier on real-world e-mails, which you can download from here. Each training e-mail is stored in a separate file. The names of spam training e-mails start with spam, while the names of ham e-mails start with ham.

## Marking criteria

Part 1 (40%):
    - Your program classifies the testing set with an accuracy significantly higher than random within 30 minutes
    - Use very simple data preprocessing so that the emails can be read into the Naive Bayes (remove everything else other than words from emails)
    - Write simple Naive Bayes multinomial classifier or use an implementation from a library of your choice
    - Classify the data
    - Report your results with a metric (e.g. accuracy) and method (e.g. cross validation) of your choice
    - Choose a baseline and compare your classifier against it

Part 2 (30%):
    - Use some smart feature processing techniques to improve the classification results
    - Compare the classification results with and without these techniques
    - Analyse how the classification results depend on the parameters (if available) of chosen techniques
    - Compare (statistically) your results against any other algorithm of your choice (use can use any library); compare and contrast results, ensure fair comparison

Part 3 (30%):
    - Calibration (15%): calibrate Naive Bayes probabilities, such that they result in low mean squared error
    - Naive Bayes extension (15%): modify the algorithm in some interesting way (e.g. weighted Naive Bayes)
    

** Convert from .ipynb to .py : ** $ ipython nbconvert --to python filter.ipynb

** BeautifulSoup: ** $ pip install beautifulsoup4




In [1]:
# Import all necessary packages
import math
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import email
import nltk



import os, re




** Part 1 Filtering **

In [2]:

# Removes header from email if present

def get_body(msg):
    if isinstance(msg, str):
        return msg
    elif msg.is_multipart():
        payloads = ''
        for payload in msg.get_payload():
            # if payload.is_multipart(): ...
            payloads += ' ' + get_body(payload)
        return payloads
    else:
        return msg.get_payload()
    
    
# Returns text containing only words (lowercase)
def remove_extras(text):
    brackets = "\([^)]*\)"
    email_address = "([\w.-]+)@([\w.-]+)"
    web_address = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    numbers = "\d+"
    alphanumeric = "([^\s\w]|_)+"
    whitespace = "\s+"

    text = re.sub(email_address, '', text)
    text = re.sub(web_address, '', text)
    text = re.sub(numbers, '', text)
    text = re.sub(alphanumeric, '', text)
    text = re.sub(whitespace, ' ', text).strip()
    text = text.lower() # Lowercase
    return text

def simple_html_filter(html_doc):

    soup = BeautifulSoup(str(html_doc), "html.parser")


    for script in soup(["script", "style"]):
        script.extract()    # rip it out
        
    text = soup.get_text()
    return text

def filter_stop_words(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

def filter_part1(email): 
    message = get_body(email)
    message = simple_html_filter(message)
    message = remove_extras(message)
    return message


# ---------------------

# Example

with open('training_data/ham000.txt') as f:
    text_test = f.read()

msg = email.message_from_string(text_test)

filter_part1(msg)



u'you are receiving this email because you signed up to receive one of our free reports if you would prefer not to receive messages of this type please unsubscribe by following the instructions at the bottom of this message dear investor thank you again for requesting our free special report the one stock that keeps wall street buzzing we began the motley fool in with the idea that investors like you deserved better better than wall streets alltoooften biased research better than analysts who speak in secret codes allowing them to hedge or spin any recommendation and better than what passes for full financial disclosure in big business today given a level playing field we believe that regular folks like us and you can do quite well in the stock market why put trust in conflicted information from others when you could count on your own abilities and potentially blow the pros away more than two million people visit our foolcom web site each month we spend a great deal of time at foolcom 

** Part 2 filtering **

In [3]:
from __future__ import division

email_address = "([\w.-]+)@([\w.-]+)"
web_address = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
numbers = "\d+"
alphanumeric = r'[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]'
whitespace = "\s+"
exclam = r'[!]'

# Returns text containing only words (lowercase)
def remove_extras2(text):
    text = re.sub(email_address, ' ' + str(hash('email')) + ' ', text)
    text = re.sub(web_address, ' ' + str(hash('web')) + ' ', text)
    text = re.sub(numbers, ' ' + str(hash('number')) + ' ', text)
    text = re.sub(alphanumeric, ' ' + str(hash('alphanumeric')) + ' ', text)
    text = re.sub(whitespace, ' ', text).strip()
    text = text.lower() # Lowercase
    return text

# Threshold between 0 and 1
def regex_ratio(string, regex):
    num_of_chars = len(string)
    num_of_chars_regex = len(re.findall(regex, string.encode('utf-8')))
    ratio = num_of_chars_regex / num_of_chars
    return ratio

def get_subject(msg):
    if isinstance(msg['subject'], str):
        return msg['subject']
    

# Example 
with open('training_data/ham000.txt') as f:
    text_test = f.read()

# FEATURES
msg = email.message_from_string(text_test)

regex_ratio(get_body(msg), alphanumeric)
regex_ratio(get_body(msg), numbers)
regex_ratio(get_body(msg), web_address)
regex_ratio(get_body(msg), email_address)
regex_ratio(get_body(msg), whitespace)
regex_ratio(get_body(msg), whitespace)
regex_ratio(get_body(msg), exclam)
subject = get_subject(msg)

# print get_body(msg)



You are receiving this email because you signed up to 
receive one of our free reports. If you would prefer 
not to receive messages of this type, please 
unsubscribe by following the instructions at the 
bottom of this message.


Dear Investor,

Thank you again for requesting our free special 
report, "The One Stock that Keeps Wall Street 
BUZZING."

We began The Motley Fool in 1993 with the idea that 
investors like you deserved better. 

Better than Wall Street's all-too-often biased 
research...  Better than analysts who speak in "secret 
codes" allowing them to hedge or spin any 
recommendation...  and better than what passes for "full 
financial disclosure" in big business today.

Given a level playing field, we believe that regular 
folks -- like us, and you -- can do quite well in the 
stock market. Why put trust in conflicted information 
from others when you could count on your own 
abilities, and potentially blow the pros away? 

More than two million people visit our Fool.c

In [4]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer 
from geoip import geolite2

def do_stemming(text):
    stemmed = ""
    for word in text.split():
#         stemmed += PorterStemmer().stem(word) + " "
        stemmed += LancasterStemmer().stem(word) + " "
    return stemmed
    
def get_source_country(email):
    ips = re.findall( r'[0-9]+(?:\.[0-9]+){3}', str(email) )

    if len(ips) > 0:
        match = geolite2.lookup(ips[0])
        if match:
            return str(match.country)
    return ''

def filter_part2(email):
    message = get_body(email)
    message = simple_html_filter(message)
    message = remove_extras2(message)
    message = filter_stop_words(message)
    message = do_stemming(message)
    return message

# ---------------------

# Example

with open('training_data/spam036.txt') as f:
    text_test = f.read()

msg = email.message_from_string(text_test)


# print(filter_part2(msg))

read in all documents and convert to a bag of words

In [5]:
import glob
training_data = []
training_labels = []
training_filenames = []
training_label_names = ['ham', 'spam']
        
# ham
for filename in glob.glob('training_data/ham*.txt'):
    f = open(filename, 'r')
    training_data.append(f.read())
    training_labels.append(0)
    training_filenames.append(filename)

# spam    
for filename in glob.glob('training_data/spam*.txt'):
    f = open(filename, 'r')
    training_data.append(f.read())
    training_labels.append(1)
    training_filenames.append(filename)


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.base import BaseEstimator

class PreProcessor(BaseEstimator):
    def __init__(self, part_name):
        self.part_name = part_name
        
    def transform(self, X):
        out = []
        for item in X:
            if self.part_name == 'part1':
                data = filter_part1(email.message_from_string(item))
            elif self.part_name == 'part2':
                data = filter_part2(email.message_from_string(item))
            else:
                raise Exception('unknwon part name for PreProcessor') 
            out.append(data)
        return out

    def fit(self, X, y=None):
        return self
    
class ads(BaseEstimator):
        
#     def __init__(self, key):
#         self.key = key
        
    def transform(self, X):
        out = []
        for i,vector in enumerate(X):
            print i
        return out

    def fit(self, X, y=None):
        return self

In [7]:
clf_pipe_p1 = Pipeline([('preprocess', PreProcessor('part1')),
                    ('vect', CountVectorizer(decode_error='ignore')),
                    ('feat', ads()),
                    ('clf', MultinomialNB()),
])

Let's test using Stratified Cross Validation:

In [ ]:
def printIncorrectClassifications(ground_truths, predictions, names):
    for i, prediction in enumerate(predictions):
        if (ground_truths[i] != prediction):
            print names[i] + ' classified as ' + training_label_names[prediction] + ' when it was ' + training_label_names[ground_truths[i]]

In [ ]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score

k = 10
skf = StratifiedKFold(training_labels, n_folds=k)
acc_per_fold = []

for train_indices, test_indices in skf:
    clf = clf_pipe_p1.fit([training_data[i] for i in train_indices], [training_labels[i] for i in train_indices])
    predictions['stratified-nb-p1'] = clf.predict([training_data[i] for i in test_indices])
    predictions['stratified-nb-p1-prob'] = clf.predict_proba([training_data[i] for i in test_indices])
    acc_per_fold.append( accuracy_score([training_labels[i] for i in test_indices], predictions['stratified-nb-p1']) )
    printIncorrectClassifications([training_labels[i] for i in test_indices], predictions['stratified-nb-p1'], [training_filenames[i] for i in test_indices])

print "Accuracy per fold:", acc_per_fold
print "Average accuracy:", np.mean(acc_per_fold)

And we'll also test using a separate (large) batch of test data

In [ ]:
import random
def pickTestData(count=1):
    labelsFile = open('testdata.label', 'r')
    labeling = labelsFile.readlines()
    random.shuffle(labeling)
    sample = labeling[0:count]
    return [open('test_data/'+row.split()[1], 'r').read() for row in sample], [1-int(row.split()[0]) for row in sample], ['test_data/'+row.split()[1] for row in sample]

test_data, test_labels, test_filenames = pickTestData(1000)

In [ ]:

from sklearn.naive_bayes import MultinomialNB
# we're using multinomial because it's most relevant for word counts

predictions['test-nb-p1'] = clf_pipe_p1.fit(training_data, training_labels).predict(test_data)
predictions['test-nb-p1-prob'] = clf_pipe_p1.fit(training_data, training_labels).predict_proba(test_data)

accuracy = accuracy_score(test_labels, predictions['test-nb-p1'])
printIncorrectClassifications(test_labels, predictions['test-nb-p1'], test_filenames)
print "Accuracy against test data:", accuracy

In [ ]:
%matplotlib inline

import itertools
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plotConfusionMatrices(ground_truths, predicted):
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(ground_truths, predicted)
    np.set_printoptions(precision=2)

    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=training_label_names,
                          title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=training_label_names, normalize=True,
                          title='Normalized confusion matrix')

    plt.show()

In [ ]:
from sklearn import metrics
import pandas as pd
from ggplot import *
from sklearn.metrics import roc_curve, auc

def plotRocSimple(ground_truths, predictions):
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(2):
        fpr[i], tpr[i], _ = roc_curve([item for item in ground_truths], [item[i] for item in predictions])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(ground_truths, [item[i] for item in predictions])
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.figure()
    lw = 2
    plt.plot(fpr[2], tpr[2], color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    fpr, tpr, _ = metrics.roc_curve(ground_truths, predictions)

    df = pd.DataFrame(dict(fpr=fpr, tpr=tpr))
    auc_str = metrics.auc(fpr,tpr)

    return ggplot(df, aes(x='fpr', y='tpr')) +\
        geom_area(alpha=0.2) +\
        geom_line(aes(y='tpr')) +\
        ggtitle("ROC Curve w/ AUC=%s" % str(auc_str)) +\
        geom_abline(linetype='dashed')

from scipy import interp
from itertools import cycle


def plotProbaROC(ground_truths, predicted, predicted_proba):
    y_test_predictions = predicted
    y_test = np.array(ground_truths)
    
    n_classes = 2
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test, [row[i] for row in predicted_proba])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), predicted_proba.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    lw = 2
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(training_label_names[i], roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Some extension of Receiver operating characteristic to multi-class')
#     plt.legend(loc="lower right")
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
plotConfusionMatrices(test_labels, predictions['test-nb-p1'])
plotProbaROC(test_labels, predictions['test-nb-p1'], predictions['test-nb-p1-prob'])

we'll use a DummyClassifier as our baseline, in stratified cross validation:

In [ ]:
from sklearn.dummy import DummyClassifier

clf_pipe_dummy = Pipeline([('preprocess', PreProcessor('part1')),
                     ('vect', CountVectorizer(decode_error='ignore')),
#                      ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', DummyClassifier('uniform')),
])

In [ ]:
k = 10
skf = StratifiedKFold(training_labels, n_folds=k)
acc_per_fold = []

for train_indices, test_indices in skf:
    dclf = clf_pipe_dummy.fit([training_data[i] for i in train_indices], [training_labels[i] for i in train_indices])
    predictions['stratified-dummy-p1'] = dclf.predict([training_data[i] for i in test_indices])
    acc_per_fold.append( accuracy_score([training_labels[i] for i in test_indices], predictions['stratified-dummy-p1']) )

print "Baseline accuracy per fold:", acc_per_fold
print "Baseline average accuracy:", np.mean(acc_per_fold)

and against our test data:

In [ ]:
predictions['test-dummy-p1'] = clf_pipe_dummy.fit(training_data, training_labels).predict(test_data)
predictions['test-dummy-p1-prob'] = clf_pipe_dummy.fit(training_data, training_labels).predict_proba(test_data)

dummy_accuracy = accuracy_score(test_labels, predictions['test-dummy-p1'])

print "Baseline accuracy against test data:", dummy_accuracy

In [ ]:
plotConfusionMatrices(test_labels, predictions['test-dummy-p1'])
plotProbaROC(test_labels, predictions['test-dummy-p1'], predictions['test-dummy-p1-prob'])

# Part 2

now we can do some more interesting things

In [ ]:
clf_pipe_p2 = Pipeline([('preprocess', PreProcessor('part2')),
                     ('vect', CountVectorizer(decode_error='ignore', binary=True, max_features=10000)),
                     ('clf', MultinomialNB(alpha=1, fit_prior=False)),
])

In [ ]:
k = 10
skf = StratifiedKFold(training_labels, n_folds=k)
acc_per_fold = []

for train_indices, test_indices in skf:
    clf = clf_pipe_p2.fit([training_data[i] for i in train_indices], [training_labels[i] for i in train_indices])
    predictions['stratified-nb-p2'] = clf.predict([training_data[i] for i in test_indices])
    acc_per_fold.append( accuracy_score([training_labels[i] for i in test_indices], predictions['stratified-nb-p2']) )

print "Accuracy per fold:", acc_per_fold
print "Average accuracy:", np.mean(acc_per_fold)

In [ ]:
predictions['test-nb-p2'] = clf_pipe_p2.fit(training_data, training_labels).predict(test_data)
predictions['test-nb-p2-prob'] = clf_pipe_p2.fit(training_data, training_labels).predict_proba(test_data)

accuracy = accuracy_score(test_labels, predictions['test-nb-p2'])

print "Accuracy against test data:", accuracy

In [ ]:
plotConfusionMatrices(test_labels, predictions['test-nb-p2'])
plotProbaROC(test_labels, predictions['test-nb-p2'], predictions['test-nb-p2-prob'])

we need to compare our part 2 algorithm to another algorithm of our choice. We'll use an SVM.

In [ ]:
from sklearn.linear_model import SGDClassifier
clf_pipe_svm = Pipeline([('preprocess', PreProcessor('part2')),
                    ('vect', CountVectorizer(decode_error='ignore')),
                    ('clf', SGDClassifier(loss='log', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])


predictions['test-svm-p2'] = clf_pipe_svm.fit(training_data, training_labels).predict(test_data)
predictions['test-svm-p2-prob'] = clf_pipe_svm.fit(training_data, training_labels).predict_proba(test_data)

accuracy = accuracy_score(test_labels, predictions['test-svm-p2'])

print "SVM accuracy against test data:", accuracy

In [ ]:
plotConfusionMatrices(test_labels, predictions['test-svm-p2'])
plotProbaROC(test_labels, predictions['test-svm-p2'], predictions['test-svm-p2-prob'])

best SVM tuned parameters

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__alpha': (1e-2, 1e-3),
}

gs_clf = GridSearchCV(clf_pipe_svm, parameters, n_jobs=-1)

gs_clf = gs_clf.fit(training_data, training_labels)

print "Best score:", gs_clf.best_score_

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

I want to tune the part2 parameters to see what the best value can be

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'vect__binary': (True, False),
              'vect__max_features': [None, 10000],
              'vect__stop_words': ('english', None),
              'clf__alpha': (0, 1),
              'clf__fit_prior': (False, True)
}

gs_clf = GridSearchCV(clf_pipe_p2, parameters, n_jobs=-1)

gs_clf = gs_clf.fit(training_data, training_labels)

print "Best score:", gs_clf.best_score_

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

# Part 3